In [1]:
import pandas as pd
import datetime as dt
import locale
import holidays

In [2]:
# --- Configurar los días festivos para Colombia para 2025:
co_holidays = holidays.CountryHoliday('CO', years=[2025])

In [4]:
ruta ="dataset/Biometrico Norte 07.2025.xls"
df_bq = pd.read_excel(ruta)

WARNING *** file size (40548) not 512 + multiple of sector size (512)


In [5]:
df_bq.isnull().sum()

Número              0
Nombre              0
Tiempo              0
Estado              0
Dispositivos        0
Tipo de Registro    0
dtype: int64

In [6]:
df_bq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Número            188 non-null    int64 
 1   Nombre            188 non-null    object
 2   Tiempo            188 non-null    object
 3   Estado            188 non-null    object
 4   Dispositivos      188 non-null    object
 5   Tipo de Registro  188 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 8.9+ KB


In [7]:
# Manejar por buenas practicas titulos en minuscula y sin espacios en blanco
new_name =[] 
for column in df_bq.columns:
    minus = column.lower()
    replace = minus.replace(" ","_")
    new_name.append(replace)

df_bq.columns = new_name


df_bq.duplicated().sum()


np.int64(0)

In [8]:
#cambia el nombre de la columna de numero a ID del trabajador
df_bq.rename(columns={'número': 'id'}, inplace=True) 

In [9]:
# Reemplaza el espacio no-breaking por un espacio normal
df_bq['tiempo'] = df_bq['tiempo'].str.replace('\xa0', ' ', regex=False)  

# Reemplaza a.m a AM para que datetime lo lea 
df_bq['tiempo'] = df_bq['tiempo'].str.replace('a. m.', 'AM', regex=False, case=False) 

# Reemplaza p.m a PM para que datetime lo lea 
df_bq['tiempo'] = df_bq['tiempo'].str.replace('p. m.', 'PM', regex=False, case=False) 



df_bq

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro
0,80794081,YEIS MUÑOZ,01/07/2025 8:03:23,Entrada,norte,0
1,80794081,YEIS MUÑOZ,01/07/2025 12:00:40,Entrada,norte,0
2,80794081,YEIS MUÑOZ,01/07/2025 12:59:53,Entrada,norte,0
3,80794081,YEIS MUÑOZ,01/07/2025 17:35:59,Entrada,norte,0
4,80794081,YEIS MUÑOZ,02/07/2025 6:57:30,Entrada,norte,0
...,...,...,...,...,...,...
183,101023549,JHON MULATO,30/07/2025 14:41:21,Salida,norte,0
184,101023549,JHON MULATO,30/07/2025 17:35:19,Salida,norte,0
185,101023549,JHON MULATO,31/07/2025 8:37:28,Salida,norte,0
186,101023549,JHON MULATO,31/07/2025 13:05:24,Salida,norte,0


In [12]:
df_bq['tiempo'] = pd.to_datetime(df_bq['tiempo'], dayfirst=True, errors='coerce')

# Crear columnas adicionales
df_bq['fecha'] = df_bq['tiempo'].dt.date
df_bq['hora'] = df_bq['tiempo'].dt.time
df_bq['mes'] = df_bq['tiempo'].dt.month


In [13]:
# Extraer la hora (en formato 24 horas)
df_bq['hora_entera'] = df_bq['tiempo'].dt.hour

# Extraer los minutos
df_bq['minutos_enteros'] = df_bq['tiempo'].dt.minute

# Calcular la fracción de la hora (como en Excel)
df_bq['fraccion_dia_hora'] = (df_bq['hora_entera'] + df_bq['minutos_enteros'] / 60) / 24


In [14]:
# Crear una lista de nombres de días de la semana en español para mapear
dias_semana_espanol = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]

# Extraer el número del día de la semana (0=lunes, 6=domingo)
#    Lo aplicamos a la parte de la fecha de la columna 'tiempo'
df_bq['dia_Numero'] = df_bq['tiempo'].dt.weekday

# Mapear el número a su nombre correspondiente en español
df_bq['dia'] = df_bq['dia_Numero'].map(lambda x: dias_semana_espanol[x])


# Busqueda de dias festivos
df_bq['festivo'] = df_bq['fecha'].apply(lambda x: x in co_holidays)

df_bq.info()
df_bq.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 188 non-null    int64         
 1   nombre             188 non-null    object        
 2   tiempo             188 non-null    datetime64[ns]
 3   estado             188 non-null    object        
 4   dispositivos       188 non-null    object        
 5   tipo_de_registro   188 non-null    int64         
 6   fecha              188 non-null    object        
 7   hora               188 non-null    object        
 8   mes                188 non-null    int32         
 9   hora_entera        188 non-null    int32         
 10  minutos_enteros    188 non-null    int32         
 11  fraccion_dia_hora  188 non-null    float64       
 12  dia_Numero         188 non-null    int32         
 13  dia                188 non-null    object        
 14  festivo   

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro,fecha,hora,mes,hora_entera,minutos_enteros,fraccion_dia_hora,dia_Numero,dia,festivo
0,80794081,YEIS MUÑOZ,2025-07-01 08:03:23,Entrada,norte,0,2025-07-01,08:03:23,7,8,3,0.335417,1,martes,False
1,80794081,YEIS MUÑOZ,2025-07-01 12:00:40,Entrada,norte,0,2025-07-01,12:00:40,7,12,0,0.500000,1,martes,False
2,80794081,YEIS MUÑOZ,2025-07-01 12:59:53,Entrada,norte,0,2025-07-01,12:59:53,7,12,59,0.540972,1,martes,False
3,80794081,YEIS MUÑOZ,2025-07-01 17:35:59,Entrada,norte,0,2025-07-01,17:35:59,7,17,35,0.732639,1,martes,False
4,80794081,YEIS MUÑOZ,2025-07-02 06:57:30,Entrada,norte,0,2025-07-02,06:57:30,7,6,57,0.289583,2,miércoles,False


In [15]:
df_bq['fecha'] = pd.to_datetime(df_bq['fecha'])

# Agrupar por persona y día, y obtener la primera y última marca
horarios = df_bq.groupby(['nombre', 'fecha'])['tiempo'].agg(['min', 'max']).reset_index()
horarios.rename(columns={'min': 'hora_entrada', 'max': 'hora_salida'}, inplace=True)

horarios['horas_trabajadas'] = (horarios['hora_salida'] - horarios['hora_entrada']).dt.total_seconds() / 3600


In [16]:
horarios['mes'] = horarios['fecha'].dt.month

resumen = horarios.groupby(['nombre', 'mes'])['horas_trabajadas'].sum().reset_index()
resumen.rename(columns={'horas_trabajadas': 'total_horas_efectivas'}, inplace=True)


In [18]:
resumen['horas_esperadas'] = 229
resumen['porcentaje_cumplimiento'] = resumen['total_horas_efectivas'] / resumen['horas_esperadas'] * 100
resumen

,nombre,mes,total_horas_efectivas,horas_esperadas,porcentaje_cumplimiento
0,JHON MULATO,7,204.117500,229,89.134279
1,YEIS MUÑOZ,7,221.526944,229,96.736657


In [19]:
df_bq.head()

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro,fecha,hora,mes,hora_entera,minutos_enteros,fraccion_dia_hora,dia_Numero,dia,festivo
0,80794081,YEIS MUÑOZ,2025-07-01 08:03:23,Entrada,norte,0,2025-07-01,08:03:23,7,8,3,0.335417,1,martes,False
1,80794081,YEIS MUÑOZ,2025-07-01 12:00:40,Entrada,norte,0,2025-07-01,12:00:40,7,12,0,0.500000,1,martes,False
2,80794081,YEIS MUÑOZ,2025-07-01 12:59:53,Entrada,norte,0,2025-07-01,12:59:53,7,12,59,0.540972,1,martes,False
3,80794081,YEIS MUÑOZ,2025-07-01 17:35:59,Entrada,norte,0,2025-07-01,17:35:59,7,17,35,0.732639,1,martes,False
4,80794081,YEIS MUÑOZ,2025-07-02 06:57:30,Entrada,norte,0,2025-07-02,06:57:30,7,6,57,0.289583,2,miércoles,False


In [22]:
horarios.head(27)

,nombre,fecha,hora_entrada,hora_salida,horas_trabajadas,mes
0,JHON MULATO,2025-07-01,2025-07-01 07:01:14,2025-07-01 16:40:28,9.653889,7
1,JHON MULATO,2025-07-02,2025-07-02 08:05:17,2025-07-02 17:33:20,9.467500,7
2,JHON MULATO,2025-07-03,2025-07-03 08:09:21,2025-07-03 14:03:00,5.894167,7
3,JHON MULATO,2025-07-04,2025-07-04 08:05:06,2025-07-04 17:33:47,9.478056,7
4,JHON MULATO,2025-07-05,2025-07-05 13:06:01,2025-07-05 13:06:01,0.000000,7
5,JHON MULATO,2025-07-07,2025-07-07 06:59:43,2025-07-07 16:36:05,9.606111,7
6,JHON MULATO,2025-07-08,2025-07-08 06:59:25,2025-07-08 16:48:46,9.822500,7
7,JHON MULATO,2025-07-09,2025-07-09 07:04:56,2025-07-09 16:48:24,9.724444,7
8,JHON MULATO,2025-07-10,2025-07-10 07:03:08,2025-07-10 16:33:36,9.507778,7
9,JHON MULATO,2025-07-11,2025-07-11 07:03:06,2025-07-11 17:33:06,10.500000,7


In [23]:
horarios.tail(27)

,nombre,fecha,hora_entrada,hora_salida,horas_trabajadas,mes
27,YEIS MUÑOZ,2025-07-01,2025-07-01 08:03:23,2025-07-01 17:35:59,9.543333,7
28,YEIS MUÑOZ,2025-07-02,2025-07-02 06:57:30,2025-07-02 16:48:21,9.847500,7
29,YEIS MUÑOZ,2025-07-03,2025-07-03 07:01:46,2025-07-03 16:33:04,9.521667,7
30,YEIS MUÑOZ,2025-07-04,2025-07-04 06:59:19,2025-07-04 16:56:27,9.952222,7
31,YEIS MUÑOZ,2025-07-05,2025-07-05 07:55:15,2025-07-05 07:55:15,0.000000,7
32,YEIS MUÑOZ,2025-07-07,2025-07-07 06:58:46,2025-07-07 13:22:16,6.391667,7
33,YEIS MUÑOZ,2025-07-08,2025-07-08 08:53:55,2025-07-08 17:37:15,8.722222,7
34,YEIS MUÑOZ,2025-07-09,2025-07-09 08:21:33,2025-07-09 17:41:10,9.326944,7
35,YEIS MUÑOZ,2025-07-10,2025-07-10 07:50:32,2025-07-10 17:41:20,9.846667,7
36,YEIS MUÑOZ,2025-07-11,2025-07-11 07:54:54,2025-07-11 17:41:01,9.768611,7
